# CP experiment 
Use this jupyter to generate the image stimuli matrix

Goal :
    We need 1000 chunks of stimuli EEG data 
    So, we need to stimuli 0-9 for 500 times which means that
    
    0 - display 50 times
    1 - display 50 times
    .
    .
    9 - display 50 times

Information:
- EEG sampling = 125 Hz with 16 electrod nodes
    
    
Requirements:

1. Genterate represent number 0-9 with 500 number
2. Divide it in to 25 blcok, 20 images per block
3. Current trial should not same number with previous


In [10]:
import random
import pylsl
import numpy as np
import pandas as pd
import time
import itertools
import math
import psychopy 
from psychopy import visual, core, event
from datetime import datetime
from IPython.display import clear_output
# python3.7-dev and libpython3.7-dev # use appropriate Python version here 
# sudo apt install dpkg-dev build-essential freeglut3-dev libgl1-mesa-dev libglu1-mesa-dev libgstreamer-plugins-base1.0-dev libgtk-3-dev libjpeg-dev libnotify-dev libpng-dev libsdl2-dev libsm-dev libtiff-dev libwebkit2gtk-4.0-dev libxtst-dev

In [11]:
#==============================================
# experiment parameters
#==============================================
total_image      = 500
num_trial        = 20
num_block        = int(total_image/num_trial)

stim_time        = 0.5
stim_blink_time  = 0.3
fixation_time    = 5  # inter trial interval, i.e., how long the fixation will stay in second


experiment_time  = (total_image * stim_time ) + ( (total_image-1)*stim_blink_time) + (num_block-1)*fixation_time
print(f"Total experiment time = {'{:.2f}'.format(experiment_time/60)} Minute" )
      
    
#==============================================
# Configuration
#==============================================
image_folder='numerical_stimuli'


#name, type, channel_count, sampling rate, channel format, source_id
#info = StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'int32', 'CytonMarkerID')#make an outlet
info = pylsl.StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'string', 'CytonMarkerID')#make an outlet
outlet = pylsl.StreamOutlet(info)
# %whos



Total experiment time = 8.66 Minute


In [12]:

def genNumbers(num):
    trial_sequence = []
    for loop in range(int(num/10)):
        sequence = np.arange(10)
        trial_sequence.extend(sequence)
    random.shuffle(trial_sequence)
    return trial_sequence


# find the 2 consecutive
def findConsecutive(data):
    for loop_i in range(len(data)):
        if loop_i <= (len(data)-2) :
            if data[loop_i] == data[loop_i+1] :
                fixConsecutive(data, loop_i+1)

# find the positon that can swap
def fixConsecutive(data, position):
    if position <= ( len(data) -2 ) :
        counter = len(data) - position
        for ii in range(counter) :
            if data[position] != data[position+ii] :
                # swap
                swapPositions(data,position, position+ii )
                return
            
# swap 2 value
def swapPositions(list, pos1, pos2): 
    list[pos1], list[pos2] = list[pos2], list[pos1] 
#     print(f"swap index {pos1} <-> {pos2}")
    return 


# check 2 consecutive value
def checkConsecutive(data):
    num_consecutive = 0
    for loop_i in range(len(data)):
        if loop_i <= (len(data)-2) :
            if data[loop_i] == data[loop_i+1] :
#                 print(f"Consecutive found : {loop_i}<->{loop_i+1} , value {data[loop_i]} " )
                num_consecutive = num_consecutive+1
    return num_consecutive


In [13]:
# generate 0-9 for 1000 value
trials = genNumbers(total_image)
# print(trials)

if checkConsecutive(trials) :
    print(f"First consecutive check : There is some consecutive number in the list")
    # find and fix consecutive number
    findConsecutive(trials)

if checkConsecutive(trials) :
    print(f"Second check : There is still two same consecutive number in the list")
    findConsecutive(trials)
else :
    print(f"Second consecutive check : PASS")

if checkConsecutive(trials) :
    print(f"Third check : There is still two same consecutive number in the list")
    findConsecutive(trials)
else :
    print(f"Third consecutive check : PASS")

blocks_imgs = np.reshape(trials, (-1,num_trial))
# blocks_imgs
    

First consecutive check : There is some consecutive number in the list
Second check : There is still two same consecutive number in the list
Third check : There is still two same consecutive number in the list


In [14]:
np.count_nonzero(blocks_imgs==0)
print(blocks_imgs)

[[5 1 8 9 7 5 8 7 8 0 5 0 9 2 9 1 9 3 7 4]
 [3 1 2 5 9 2 7 9 6 4 7 1 4 9 8 7 8 7 1 5]
 [1 5 2 3 0 6 8 5 9 5 0 9 4 6 0 1 2 4 9 3]
 [5 9 8 3 0 9 3 7 4 1 7 0 4 2 6 5 4 3 9 2]
 [7 8 4 3 0 4 3 1 3 1 5 8 3 1 9 4 7 4 6 1]
 [2 0 7 0 2 4 8 6 4 6 7 4 5 4 8 2 6 2 6 8]
 [9 7 1 2 3 5 8 5 1 3 5 0 5 7 2 4 3 1 2 0]
 [4 2 0 8 4 9 8 7 6 5 0 1 9 0 5 0 6 2 8 1]
 [0 6 2 6 8 9 4 6 1 6 7 6 4 2 7 3 2 4 1 0]
 [1 2 1 8 9 4 9 1 5 3 0 4 2 0 9 8 4 9 7 1]
 [8 6 1 7 8 2 8 7 2 7 3 4 0 2 5 0 3 8 2 3]
 [7 6 7 5 3 9 7 6 7 9 6 2 5 0 9 4 9 5 0 5]
 [3 9 7 5 1 8 6 8 1 8 1 7 0 8 1 0 9 0 2 3]
 [2 1 5 4 2 6 2 0 3 7 3 0 7 8 0 1 7 1 2 6]
 [2 1 7 9 3 2 3 9 3 2 6 1 4 7 8 0 3 7 9 0]
 [4 5 4 6 9 6 4 8 4 9 0 6 5 6 7 8 0 7 9 4]
 [6 8 2 4 3 2 3 9 2 8 3 6 8 7 5 7 9 0 1 3]
 [5 3 1 6 1 7 1 4 1 2 8 0 1 7 0 3 9 5 4 8]
 [3 0 6 7 3 2 8 7 8 6 8 7 5 4 3 5 6 5 6 0]
 [6 7 2 1 5 4 5 6 8 9 8 3 0 8 7 4 2 5 8 9]
 [0 9 3 9 0 9 4 9 2 0 8 5 7 3 1 3 0 3 4 1]
 [9 1 7 4 9 6 5 2 5 4 3 4 3 4 0 4 2 6 2 6]
 [9 4 6 8 1 6 8 6 5 1 5 2 4 3 7 0 6 8 6 7]
 [2 5 6 2 1

## Experiment section

In [15]:
def drawTextOnScreen(massage) :
    message = visual.TextStim( mywin, text=massage, languageStyle='LTR')
    message.contrast =  0.3
    message.height= 0.07
    message.draw() # draw on screen
    mywin.flip()   # refresh to show what we have draw

def drawTrial( fileName, stimTime ) :
    imgPath=image_folder+"/"+str(fileName)+".png"
    drawTextOnScreen('') 
    core.wait(stim_blink_time)
    img = visual.ImageStim( mywin,  image=imgPath )
    img.size *= 1.5
    img.draw()
    mywin.flip()
    eegMarking(fileName, "img_stim" )
    core.wait(stimTime)
    
def drawFixation(fileName, fixationTime):
    fixation = visual.ShapeStim(mywin,
                                vertices=((0, -0.5), (0, 0.5), (0,0), (-0.5,0), (0.5, 0)),
                                lineWidth=5,
                                closeShape=False,
                                lineColor="white"
            )
    fixation.draw()
    if not(isTrianing) :
        text = f"Block {block+1} / {len(blocks_imgs)}"
        message = visual.TextStim( mywin, text=text, languageStyle='LTR' )
        message.contrast =  0.3
        message.pos = (0, -0.6)
        message.draw() # draw on screen
        
    mywin.flip()   # refresh to show what we have draw
    eegMarking(fileName, "fixation" )
    core.wait(fixationTime-0.5)
    drawTextOnScreen('')
    core.wait(0.5)
     
def eegMarking(img, stampType):   # use trial variable from main
    if not isTrianing :
        if stampType == "img_stim" :
            markerString = str(block+1) + "," + str(trial) + ","  + str(img)
        elif stampType == "fixation" :
            markerString = str((block+1)*-1) + "," +str("Fixation") + "," + str("Fixation")
    else:
        markerString = 'Training'
    markerString= str(markerString)                              
    print("Marker string {}".format(markerString))
    outlet.push_sample([markerString])

### Stimulies

In [16]:
mywin = visual.Window([1366, 768], color="black", units='norm')     # set the screen and full screen mode
# mywin = visual.Window([1920, 1080], color="black", units='norm')    # set the screen and full screen mode

##############
####  Training session
while True:
    isTrianing = True
    drawTextOnScreen('Training session, Press space bar to start')
    keys = event.getKeys()
    if 'space' in keys:      # If space has been pushed
        start = time.time()
        drawTextOnScreen('') 
        trial = 1
        for img in blocks_imgs[4]:
            drawTrial(img, stim_time)   # drawTrail(fileName, stimTime, thinkingTime, fixationTime)

            if trial == 3 :
                break
            trial += 1

        drawFixation(img, fixation_time)
        drawTextOnScreen('End of training session')
        core.wait(5)
        isTrianing = False
        break

        
################
####### Experiment session
        
while True:
    drawTextOnScreen('Experiment session : Press space bar to start')
    keys = event.getKeys()
    if 'space' in keys:      # If space has been pushed
        start = time.time()
        drawTextOnScreen('') 
        
        for block in range(len(blocks_imgs)) :
            trial = 1
            for img in blocks_imgs[block]:
                clear_output(wait=True)
                drawTrial(img, stim_time)   # drawTrail(fileName, stimTime, thinkingTime, fixationTime)
                trial += 1
            if block != num_block:  # do not draw fixation on last block
                drawFixation(img, fixation_time)
        drawTextOnScreen('End of experiment, Thank you')
        
        stop  = time.time()
        print(f"Total experiment time = {(stop-start)/60} ")
        core.wait(10)
        break
    
mywin.close()

KeyboardInterrupt: 

In [22]:
mywin.close()